# PV China 2020 Dataset

This dataset maps the distribution of photovoltaic (PV) solar power installations across China as of *2020*. It was generated using a pixel-based **Random Forest classifier** applied to **Landsat 8 Surface Reflectance** products, with all processing conducted on **Google Earth Engine (GEE)**.

This dataset and its methodology are described in the following paper:

> **Huang, Y., Liu, J., Xu, Y. et al. (2022)**  
> *A high-resolution inventory of photovoltaic power installations in China.*  
> *Earth System Science Data*, **14**, 2839–2854.  
> [https://doi.org/10.5194/essd-14-2839-2022](https://doi.org/10.5194/essd-14-2839-2022)

The dataset is publicly available on **Zenodo**:
> **[PV_China_2020 on Zenodo (Record #6849477)](https://zenodo.org/record/6849477)**

# Setup

In [9]:
import os
import zipfile
import requests

## Download

In [11]:
url = "https://zenodo.org/record/6849477/files/PV_China_2020.zip?download=1"

# Output folder
output_folder = os.path.join("..", "..", "data", "raw_data", "pv_china_2020")
os.makedirs(output_folder, exist_ok=True)

zipname = "pv_china_2020.zip"
zippath = os.path.join(output_folder, zipname)

# Download the file
print("Downloading dataset...")
response = requests.get(url)
with open(zippath, "wb") as f:
    f.write(response.content)
print("✅ Downloaded:", zipname)

# Extract files
print("Extracting files...")
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

# List extracted files
print("Extracted files:", os.listdir(output_folder))


PermissionError: [WinError 5] Accès refusé: '..\\..\\data'

In [12]:
os.path.isdir("../../data")
os.access("../../data", os.W_OK)

False